In [1]:
from nsa import estimators
from nsa import utils, evaluators

from torchvision.models import resnet18, vit_b_16

/home/pat/projects/neural-subspace-analysis/nsa/estimators.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create random data
x = torch.randn(10, 3, 224, 224)
y = torch.randint(0, 1000, (10,))

# Create a TensorDataset
dataset = TensorDataset(x,y)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=2) # You can adjust the batch size

print("TensorDataset created with data shape:", dataset.tensors[0].shape)
print("DataLoader created with batch size:", dataloader.batch_size)

TensorDataset created with data shape: torch.Size([10, 3, 224, 224])
DataLoader created with batch size: 2


# CNNs (ResNet18)

In [3]:
def ano():

    model = resnet18()

    cov = estimators.estimate_cov_mat_at_layer(
        model=model,
        layer="layer1",
        dataloader=dataloader,
        device="cpu",
    )

    eigvals, eigvecs = utils.eigh(cov)

    eval_recon = evaluators.ReconstructionErrorWithLowRankProjectionEvaluator()

    df1 = eval_recon.evaluate(model=model, layer="layer1", dataloader=dataloader, U=eigvecs, arr_ks=[1, 5, 32, 64])

    eval_acc = evaluators.AccuracyWithLowRankProjectionEvaluator(num_classes=1000)
    df2 = eval_acc.evaluate(model=model, layer="layer1", dataloader=dataloader, U=eigvecs, arr_ks=[1, 5, 32, 64])

    return df1, df2

df1, df2 = ano()

[layer=layer1] estimating covariance matrix:   0%|          | 0/5 [00:00<?, ?it/s]

[layer=layer1] evaluating reconstruction error:   0%|          | 0/5 [00:00<?, ?it/s]

[layer=layer1] evaluating accuracy:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
df1

,k,norm,recon_err,cossim,d
0,1,15.076886,2.670829,0.984690,64
1,5,15.076886,2.605247,0.985291,64
2,32,15.076886,2.252571,0.988957,64
3,64,15.076886,0.000009,1.000000,64


In [5]:
df2

,k,acc,xent,d
0,1,0.0,7.136613,64
1,5,0.0,7.160556,64
2,32,0.0,7.113336,64
3,64,0.0,7.141576,64


# ViTs

In [6]:
def ano():

    model = vit_b_16(weights="IMAGENET1K_V1")
    layer = "encoder.layers.1"
    cov = estimators.estimate_cov_mat_at_layer(
        model=model,
        layer=layer,
        dataloader=dataloader,
        device="cpu",
    )

    eigvals, eigvecs = utils.eigh(cov)


    d = len(eigvals)

    eval_recon = evaluators.ReconstructionErrorWithLowRankProjectionEvaluator()

    df1 = eval_recon.evaluate(model=model, layer=layer, dataloader=dataloader, U=eigvecs, arr_ks=[1, 128, d])

    eval_acc = evaluators.AccuracyWithLowRankProjectionEvaluator(num_classes=1000)
    df2 = eval_acc.evaluate(model=model, layer=layer, dataloader=dataloader, U=eigvecs, arr_ks=[1, 128, d])

    return eigvals, df1, df2
eigvals, df1, df2 = ano()

[layer=encoder.layers.1] estimating covariance matrix:   0%|          | 0/5 [00:00<?, ?it/s]

[layer=encoder.layers.1] evaluating reconstruction error:   0%|          | 0/5 [00:00<?, ?it/s]

[layer=encoder.layers.1] evaluating accuracy:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
eigvals.shape

torch.Size([768])

In [8]:
df1

,k,norm,recon_err,cossim,d
0,1,31.340866,38.154732,-0.017518,768
1,128,31.340866,0.977067,0.999440,768
2,768,31.340866,0.000042,1.000000,768


In [9]:
df2

,k,acc,xent,d
0,1,0.0,6.769803,768
1,128,0.0,8.160949,768
2,768,0.0,8.167736,768


In [10]:
print("all passed!")

all passed!


add mlp
...

add mlp
...